<a href="https://colab.research.google.com/github/mathgurgel/TP_IA/blob/main/tp_ia.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Algoritmos de Classificação

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

from sklearn.model_selection import cross_val_score 
from sklearn.model_selection import KFold
from sklearn.model_selection import RepeatedKFold
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import RepeatedStratifiedKFold

from imblearn.over_sampling import SMOTE

from sklearn.metrics import accuracy_score, f1_score

import pandas as pd

### Google drive imports

In [ ]:
# libraries for the files in google drive
from pydrive.auth import GoogleAuth
from google.colab import drive
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

### Download files from drive

In [ ]:
def download_file(id, path):

  download = drive.CreateFile({'id': id})
  download.GetContentFile(path)

In [ ]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

download_file('', 'conjunto_de_dados/falencia-treinamento.csv')
download_file('', 'conjunto_de_dados/falencia-teste.csv')
download_file('', 'conjunto_de_dados/modelo-submissao.csv')

## Leitura dos dados

In [ ]:
#Lendo os dados
df_training = pd.read_csv('conjunto_de_dados/falencia-treinamento.csv')
df_test = pd.read_csv('conjunto_de_dados/falencia-teste.csv')
df_test_result = pd.read_csv('conjunto_de_dados/modelo-submissao.csv')

#Melhorando o resultado aumentando o tamanho dos dados de treinamento
df_training_copy = df_training.copy()
df_training_copy = df_training_copy.sample(frac=1).reset_index(drop=True)
df = pd.concat([df_training, df_training_copy], ignore_index=True, sort=False)

#Aumentando base de dados utilizando a técnica SMOTE para aumentar a classe minoritária
X = df.drop('Resultado', axis='columns')
y = df['Resultado']

smote = SMOTE(random_state=42)
X_res, y_res = smote.fit_resample(X, y)
df_res = pd.concat([X_res, y_res], axis=1)

Mounted at /content/drive


## Pre processing

In [ ]:
X_res = df_res.drop('Resultado', axis='columns')
y_res = df_res['Resultado']

#Nova divisão treino e teste para tentar melhorar os resultados
#kf = KFold(n_splits=5, shuffle=True, random_state=10)
kf = RepeatedKFold(n_splits=10, n_repeats=10, random_state=42)
#kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=10)
#kf =RepeatedStratifiedKFold(n_splits=5, n_repeats=5, random_state=10)
for train_index, test_index in kf.split(df_res):
    # Split data into train and test sets
    X_train, X_test = X_res.iloc[train_index], X_res.iloc[test_index]
    y_train, y_test = y_res.iloc[train_index], y_res.iloc[test_index]

## Logistic Regression

In [ ]:
def logistic_regression():
    # Separa as variáveis independentes e dependentes (treinamento)
    #independent_variables_training = df_training.drop('Resultado', axis='columns')
    #dependent_variable_training = df_training['Resultado']

    # Recebe as variáveis independentes do conjunto de teste
    #independent_variables_test = df_test

    independent_variables_training = X_train
    dependent_variable_training = y_train

    independent_variables_test = X_test    

    # Cria o modelo de regressão logística
    logistic_regression = LogisticRegression(C=0.01, random_state=42)

    # Treina o modelo
    logistic_regression.fit(independent_variables_training, dependent_variable_training)

    # Faz a previsão do modelo
    prediction = logistic_regression.predict(independent_variables_test) # Resultado do modelo

    # Calcula a acurácia do modelo
    #accuracy = accuracy_score(df_test_result['Resultado'], prediction)
    #f_measure = f1_score(df_test_result['Resultado'], prediction)

    accuracy = accuracy_score(y_test, prediction)
    f_measure = f1_score(y_test, prediction)

    print('Logistic Regression')
    print('Acurácia: ', accuracy)
    print('F-Measure: ', f_measure)

## Decision Tree

In [ ]:
def decision_tree():
    # Separa as variáveis independentes e dependentes (treinamento)
    #independent_variables_training = df_training.drop('Resultado', axis='columns')
    #dependent_variable_training = df_training['Resultado']

    # Recebe as variáveis independentes do conjunto de teste
    #independent_variables_test = df_test

    independent_variables_training = X_train
    dependent_variable_training = y_train

    independent_variables_test = X_test 

    # Cria o modelo de Decision Tree
    decision_tree = DecisionTreeClassifier(random_state=10)

    # Treina o modelo
    decision_tree.fit(independent_variables_training, dependent_variable_training)

    # Faz a previsão do modelo
    prediction = decision_tree.predict(independent_variables_test) # Resultado do modelo

    # Calcula a acurácia do modelo
    #accuracy = accuracy_score(df_test_result['Resultado'], prediction)
    #f_measure = f1_score(df_test_result['Resultado'], prediction)
    
    accuracy = accuracy_score(y_test, prediction)
    f_measure = f1_score(y_test, prediction)

    print('Decision Tree')
    print('Acurácia: ', accuracy)
    print('F-Measure: ', f_measure)

## Random Forest

In [ ]:
def random_forest():
    # Separa as variáveis independentes e dependentes (treinamento)
    #independent_variables_training = df_training.drop('Resultado', axis='columns')
    #dependent_variable_training = df_training['Resultado']

    # Recebe as variáveis independentes do conjunto de teste
    #independent_variables_test = df_test

    independent_variables_training = X_train
    dependent_variable_training = y_train

    independent_variables_test = X_test

    # Cria o modelo de Random Forest
    random_forest = RandomForestClassifier(random_state=10)

    # Treina o modelo
    random_forest.fit(independent_variables_training, dependent_variable_training)

    # Faz a previsão do modelo
    prediction = random_forest.predict(independent_variables_test) # Resultado do modelo

    # Calcula a acurácia do modelo
    #accuracy = accuracy_score(df_test_result['Resultado'], prediction)
    #f_measure = f1_score(df_test_result['Resultado'], prediction)
    
    accuracy = accuracy_score(y_test, prediction)
    f_measure = f1_score(y_test, prediction)

    print('Random Forest')
    print('Acurácia: ', accuracy)
    print('F-Measure: ', f_measure)

## Resultados

In [ ]:
logistic_regression()
decision_tree()
random_forest()

/usr/local/lib/python3.9/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Logistic Regression
Acurácia:  0.7454545454545455
F-Measure:  0.7543859649122806
Decision Tree
Acurácia:  0.9818181818181818
F-Measure:  0.9824561403508771
Random Forest
Acurácia:  0.9727272727272728
F-Measure:  0.9734513274336283
